In [19]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import os
import shutil

import representation
import kex_data
import kex_headers
import listmode
import petlink
import position
import e7tools
import filefit as ff

In [20]:
limo = kex_data.get_listmode()

In [21]:
#limo = {k: v[:100*1000] for k,v in limo.items()} #subset for testing

In [22]:
limo_recon = {k: listmode.get_fov_image_bins(v) for k,v in limo.items()}

In [23]:
toflor = {k: v['tof_lor'] for k,v in limo_recon.items()}
prompts = {k: v['tof_lor'][:, v['is_prompt']] for k,v in limo_recon.items()}

In [24]:
limo_com = {k: position.listmode_center(v['fov_image_bins']) for k,v in limo_recon.items()}

In [25]:
#move gate 1 to gate 0
ref = np.zeros(3)
translation = np.zeros(3)
limo_mix = {}
for dkey,pos in limo_com.items():
    if dkey.endswith('0'):
        ref = pos
        mix = toflor[dkey]
    else:
        translation = ref - pos #move to reference, pos + (ref - pos) = ref + 0
        translation_mm = translation*kex_headers.MM_PER_PIXEL
        moved_toflor, is_fov = representation.move_translation(prompts[dkey], translation_mm)
        mix = listmode.join_gates((mix, moved_toflor[:, is_fov]))
        mix = listmode.join_gates((mix, toflor[dkey][:, limo_recon[dkey]['is_prompt']==False]))
        if dkey.endswith('1'):
            pkey = dkey.split(" ")[0] 
            limo_mix[pkey] = mix

MemoryError: Unable to allocate 138. MiB for an array with shape (36099939,) and data type int32

In [ ]:
#convert into sinogram
shape = kex_headers.HISTOGRAM_SHAPE
mc_sinograms = {k: listmode.get_histogram(v, shape, dtype=np.uint16) for k,v in limo_mix.items()}

In [ ]:
hdrs = kex_data.get_sino_hdrs()
mhdrs = kex_data.get_sino_mainhdrs()
for h,m in zip(hdrs, mhdrs):
    filling = [h,m]
    mc_dir = r"C:/Users/petct/Desktop/Hugo/Code/PythonMEX/motion_corrected"
    ff.prep_folder(mc_dir, filling)

In [ ]:
#account for 2 gates of data in gate 0 timewindow of 61 seconds
header_fix = True
if not header_fix:
    mc_sinograms = {k: v//2 for k, v in mc_sinograms.items()} #instead of double time in sinogram
else:
    header_paths = kex_data.get_sino_hdrs()
    header_paths = ff.refolder_paths(paths=header_paths, folder=mc_dir)
    for path in header_paths:
        with open(path, 'rt') as f:
            lines = f.readlines()
            for k, line in enumerate(lines):
                print(line[:-1])
                keys = ["!image duration (sec):=60", "%image duration from timing tags (msec):=60014"]
                for key in keys:
                    if key in line:
                        lines[k] = line.replace("60", str(2*60))
                        print("------to do: double time at k={}".format(k))
                        print(lines[k])
                        break
        with open(path, 'wt') as f:
            for line in lines:
                f.write(line)

In [ ]:
#check before saving and reconstructing
fig, axes = plt.subplots(ncols=2, figsize=(20,4))
axes = dict(zip(kex_data.PHANTOM_KEYS, axes))
for pkey, ax in axes.items():
    tof= 0
    mi = 50
    im = ax.imshow(mc_sinograms[pkey][tof, mi])
    ax.set_xlabel("transaxial angle")
    ax.set_ylabel("radial offset")
    fig.colorbar(im, ax=ax)

In [ ]:
#save to ref
ref = 0
sino_paths = kex_data.get_sino_paths()
sino_paths = ff.refolder_paths(mc_dir, sino_paths)
sino_paths = dict(zip(kex_data.DATA_KEYS, sino_paths))
for pkey in kex_data.PHANTOM_KEYS:
    dkey = pkey + " " + kex_data.GATE_KEYS[ref]
    output_path = sino_paths[dkey]
    print(output_path)
    with open(output_path, 'wb') as file:
        sino = mc_sinograms[pkey].astype('uint16')
        file.write(sino)

In [ ]:
#reconstruct with e7tools
mc_mhdrs = ff.refolder_paths(mc_dir, kex_data.get_sino_mainhdrs())
mc_mhdrs = dict(zip(kex_data.DATA_KEYS, mc_mhdrs))
for pkey,dcr in zip(kex_data.PHANTOM_KEYS, kex_data.RECON_DCR[::2]):
    dkey = pkey + " " + kex_data.GATE_KEYS[ref]
    image_path = mc_dir + "/"+pkey
    npath = kex_data.NORM_PATH
    print("return code", 
          e7tools.kex_recon(mc_mhdrs[dkey], npath, image_path, verbose=False, dcr=dcr))

In [ ]:
#get reconstructions
import filefit as ff
vfiles = []
with os.scandir(mc_dir) as it:
    for entry in it:
        if entry.is_file() and entry.name.endswith('.v'):
            print("v file", entry.name)
            vfiles.append(entry.name)
vfiles = dict(zip(kex_data.PHANTOM_KEYS, vfiles[::-1]))
mc_recon = {k: ff.get_v_data(mc_dir+"/"+v) for k,v in vfiles.items()}

In [ ]:
import volume_view as vv
importlib.reload(vv)
import visual
importlib.reload(visual)
def my_plot(img_dict, pos = [47,120,100], **imshowkwargs):
    nims = len(img_dict)
    fig, axes = plt.subplots(nrows=nims, ncols=3, figsize=(20, 7*nims))
    for (key,img), row in zip(img_dict.items(), axes):
        dimlabels = 'zyx'
        views = vv.plot_views(img, 
                      position=pos,
                      axes=row, 
                      dimlabels=dimlabels, 
                      img_title=key, **imshowkwargs)
        visual.same_colorbar(fig, views, row, **imshowkwargs)

In [ ]:
#my_plot(mc_recon, [47,120, 100], **{'clim':[0, 1600]})
my_plot(mc_recon)

In [ ]:
importlib.reload(kex_data)
ref_v_recon = kex_data.get_v_data()
_,_, dicom_paths = kex_data.paths()
print(dicom_paths)
print(ref_v_recon.keys())

In [ ]:
ref_dict = {pkey: ref_v_recon[pkey+" 0"] for pkey in kex_data.PHANTOM_KEYS}
#my_plot(ref_dict, [47,120, 100], **{'clim':[0, 1600]})
my_plot(ref_dict)